In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
x=df.iloc[:,0:-1].values
y=df.iloc[:,-1]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [6]:
x=scaler.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
x.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [10]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 2s 46ms/step - loss: 0.7461 - accuracy: 0.4967 - val_loss: 0.6898 - val_accuracy: 0.5649
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6800 - accuracy: 0.6026 - val_loss: 0.6356 - val_accuracy: 0.6494
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6315 - accuracy: 0.6661 - val_loss: 0.5960 - val_accuracy: 0.6948
Epoch 4/100
20/20 [==============================] - 0s 12ms/step - loss: 0.5948 - accuracy: 0.6938 - val_loss: 0.5680 - val_accuracy: 0.7273
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5693 - accuracy: 0.7199 - val_loss: 0.5468 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5475 - accuracy: 0.7296 - val_loss: 0.5285 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5310 - accuracy: 0.7476 - val_loss: 0.5166 - val_accuracy: 0.7597
Epoch 8/100

In [12]:
##  1.how to select appropriate optimizer
##  2.how to select number of nodes in layer
##  3.how to select number of layers
##  4.all in all one model

In [16]:
!pip install keras-tuner

import kerastuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.5 MB/s eta 0:00:00


<ipython-input-16-ee470052b9b0>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [17]:
##  1.how to select appropriate optimizer
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    ## it will make 4 models for different optimizer
    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [18]:
## it will work just like a grid search cv
tuner=kt.RandomSearch(build_model,##whatever function we created
                      objective='val_accuracy', ##what do we want to improve
                      max_trials=5) ## how many times we want to make trials

In [19]:
##search will find the best parameters
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 09s


In [20]:
## getting the best parameters
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [21]:
##getting the best models
model=tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))
##epochs=6 since it has already run 5 max trials

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5460 - accuracy: 0.7329 - val_loss: 0.5286 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5271 - accuracy: 0.7427 - val_loss: 0.5154 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5126 - accuracy: 0.7492 - val_loss: 0.5080 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5011 - accuracy: 0.7573 - val_loss: 0.4997 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4921 - accuracy: 0.7622 - val_loss: 0.4942 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4835 - accuracy: 0.7622 - val_loss: 0.4890 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4777 - accuracy: 0.7655 - val_loss: 0.4850 - val_accuracy: 0.7857
Epoch 14

In [24]:
##  2.how to select number of neurons in a layer

In [25]:
def build_model(hp):##hp->object of hyper parameter
    model=Sequential()
    ## it will take the initial limit and final limit value and stepsize.
    ##first it will set the 8 neuron in a model and then 16 neuron in another model and like that it will go upto the 128 neuron and give the best no. of neuron
    units=hp.Int('units',min_value=8,max_value=128)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [26]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='parameter tuning')

In [27]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 10s


In [28]:
tuner.get_best_hyperparameters()[0].values

{'units': 50}

In [29]:
model=tuner.get_best_models(num_models=1)[0]

In [30]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 19ms/step - loss: 0.5013 - accuracy: 0.7687 - val_loss: 0.4935 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4894 - accuracy: 0.7704 - val_loss: 0.4859 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4808 - accuracy: 0.7720 - val_loss: 0.4806 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4745 - accuracy: 0.7801 - val_loss: 0.4801 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4696 - accuracy: 0.7769 - val_loss: 0.4773 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4663 - accuracy: 0.7801 - val_loss: 0.4783 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4630 - accuracy: 0.7818 - val_loss: 0.4782 - val_accuracy: 0.7727
Epoch 14

In [31]:
def build_model(hp):
    model=Sequential()

    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                    input_dim=8
                ))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])

                ))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adalta']),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    return model

In [33]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir',
                     project_name='final1')

In [34]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))


Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 11s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 40,
 'activation0': 'sigmoid',
 'dropout0': 0.5,
 'optimizer': 'nadam',
 'units1': 64,
 'activation1': 'relu',
 'dropout1': 0.2,
 'units2': 104,
 'activation2': 'tanh',
 'dropout2': 0.6,
 'units3': 40,
 'activation3': 'tanh',
 'dropout3': 0.9,
 'units4': 56,
 'activation4': 'tanh',
 'dropout4': 0.8,
 'units5': 128,
 'activation5': 'sigmoid',
 'dropout5': 0.4,
 'units6': 80,
 'activation6': 'relu',
 'dropout6': 0.4,
 'units7': 24,
 'activation7': 'sigmoid',
 'dropout7': 0.2,
 'units8': 104,
 'activation8': 'sigmoid',
 'dropout8': 0.1,
 'units9': 48,
 'activation9': 'tanh',
 'dropout9': 0.7}

In [36]:
model=tuner.get_best_models(num_models=1)[0]

In [37]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 [==============================] - 4s 17ms/step - loss: 0.6949 - accuracy: 0.5945 - val_loss: 0.6564 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6837 - accuracy: 0.6075 - val_loss: 0.6516 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6765 - accuracy: 0.6042 - val_loss: 0.6538 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6641 - accuracy: 0.6319 - val_loss: 0.6517 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6657 - accuracy: 0.6270 - val_loss: 0.6525 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6657 - accuracy: 0.6417 - val_loss: 0.6521 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6702 - accuracy: 0.6303 - val_loss: 0.6542 - val_accuracy: 0.6429
Epoch 14